In [21]:
from lib.agents import *
import numpy as np

In [22]:
class Wall(Thing):
    pass

class Goal(Thing):
    pass

class Fringe(Thing):
    pass


In [23]:
class Maze(GraphicEnvironment):
    def percept(self, agent):
        things = self.list_things_at(agent.location)
        return things
    
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'right':
            agent.moveforward([1,0])
        elif action == 'left':
            agent.moveforward([-1,0])
        elif action == 'up':
            agent.moveforward([0,-1])
        elif action == 'down':
            agent.moveforward([0,1])
                    
    def is_done(self):
        things = []
        for agent in self.agents:
            for thing in self.list_things_at(agent.location):
                things.append(thing)
        for thing in things:
            if isinstance(thing, Goal):
                return True
        return False



In [24]:
class OurAgent(Agent):
    location = [0, 1]
    direction = Direction("down")
    
    def moveforward(self, loc):
        self.location[0] += loc[0]
        self.location[1] += loc[1]
    
    def turn(self, d):
        self.direction = Direction(d)
        
def program(percepts):
    global it
    '''Returns an action based on it's percepts'''
        
    for p in percepts:
        if isinstance(p, Wall):
            print("Cannot move in a wall\nImplement the algorithm again")
            
    it += 1
    return steps[it]

In [25]:


maze_layout = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 3, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1],
    [1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1],
    [1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1],
    [1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1],
    [1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
    [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1],
    [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1],
    [1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1],
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
]


maze_instance = Maze(20, 20, color={'Goal': (0, 128, 0), 'Wall': (165,42,42), 'OurAgent': (255, 255, 0),'Fringe': (0, 0, 255)})

Now that our maze_instance is ready for the 2D motion of our energetic dog, lets test it!

In [26]:
def heuristic(position, goal):
    """Calculate Manhattan distance heuristic."""
    return abs(position[0] - goal[0]) + abs(position[1] - goal[1])


def cost_function(current, next, maze_layout):
    """Calculate the cost of moving from current to next, considering walls."""
    if maze_layout[next[0]][next[1]] == 1:  # Wall
        return float('inf')  # High cost for walls
    else:
        dx = next[0] - current[0]
        dy = next[1] - current[1]
        if dx == 0:
            if dy == -1:
                return 2  # Up
            elif dy == 1:
                return 3  # Down
        elif dy == 0:
            if dx == -1:
                return 1  # Left
            elif dx == 1:
                return 4  # Right
        return 0


In [27]:

def neighbors(position):
    """Generate neighboring positions."""
    x, y = position
    neighboring_positions = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
    valid_neighbors = []
    for neighbor in neighboring_positions:
        if 0 <= neighbor[0] < len(maze_layout) and 0 <= neighbor[1] < len(maze_layout[0]) and maze_layout[neighbor[0]][neighbor[1]] != 1:
            valid_neighbors.append(neighbor)
    return valid_neighbors



def reconstruct_path(came_from, start, end):
    """Reconstruct the path from start to end."""
    current = end
    path = []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start)
    path.reverse()
    return path


In [28]:

from queue import PriorityQueue
def a_star_search(maze_layout, start, end):
    space_complexity=0
    time_complexity=0
    """Perform A* search to find the optimal path."""
    frontier = PriorityQueue()
    frontier.put((0,start))
    time_complexity+=1
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        current = frontier.get()[1]

        if current == end:
          break

        for next_node in neighbors(current):
            new_cost = cost_so_far[current] + cost_function(current, next_node,maze_layout)
            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                priority = new_cost + heuristic(end, next_node)
                frontier.put((priority, next_node))
                time_complexity+=1
                came_from[next_node] = current
                space_complexity+=1
    if end not in came_from:
        print('no path found from ', start, 'to', end)
        return None  # No path found
    
    path = reconstruct_path(came_from, start, end)
    # print(path)
    complexity=[time_complexity,space_complexity]
    return path,complexity


PATH_COLOR = (0, 0, 255)  # Blue



def algo(maze):
    """
    implement this function using the maze from input
    3 in the maze represents the location of the agent
    2 represents the goal
    1 represents the wall
    0 represent the free space in which our agent can walk
    return and array of moves like this e.g. ["down", "right", "up", "left", "left", "down"]
    so that the agent will be at the goal 
    use maze_instance.run(0) to show updates without running logic

    """
    
    start = (1, 1)
    end = (8,4)

    path,complexity=a_star_search(maze,start,end)

    
    for pos in path:
        x, y = pos
        fringe = Fringe()
        maze_instance.add_thing(fringe, [x, y])
        fringe.color = PATH_COLOR

    if path is None:
        print("No path found!")
        return []
    
    actions = []
    for i in range(len(path) - 1):
        current = path[i]
        next = path[i+1]
        dx = next[0] - current[0]
        dy = next[1] - current[1]
        if dx == 0:
            if dy == -1:
                actions.append("up")
            elif dy == 1:
                actions.append("down")
        elif dy == 0:
            if dx == -1:
                actions.append("left")
            elif dx == 1:
                actions.append("right")
   
    return actions,complexity
   




In [29]:
from queue import PriorityQueue


def ucs_search(maze_layout, start, end, neighbors_function):
    """Perform Uniform Cost Search to find the optimal path."""
    space_complexity=0
    time_complexity=0
    frontier = PriorityQueue()
    frontier.put((0, start))
    time_complexity+=1

    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        current_cost, current = frontier.get()

        if current == end:
            break

        for next_node in neighbors_function(current):
            new_cost = cost_so_far[current] + cost_function(current, next_node, maze_layout)
            if next_node not in cost_so_far or new_cost < cost_so_far[next_node]:
                cost_so_far[next_node] = new_cost
                priority = new_cost
                frontier.put((priority, next_node))
                time_complexity+=1
                came_from[next_node] = current
                space_complexity+=1
        
    complexity=[time_complexity,space_complexity]
    
    path = reconstruct_path(came_from, start, end)
    return path,complexity

def neighbors(position):
    """Generate neighboring positions."""
    x, y = position
    neighboring_positions = [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]
    valid_neighbors = []
    for neighbor in neighboring_positions:
        if 0 <= neighbor[0] < len(maze_layout) and 0 <= neighbor[1] < len(maze_layout[0]) and maze_layout[neighbor[0]][neighbor[1]] != 1:
            valid_neighbors.append(neighbor)
    return valid_neighbors

PATH_COLOR = (0, 0, 255)  # Blue



def algo(maze):
    """
    implement this function using the maze from input
    3 in the maze represents the location of the agent
    2 represents the goal
    1 represents the wall
    0 represent the free space in which our agent can walk
    return and array of moves like this e.g. ["down", "right", "up", "left", "left", "down"]
    so that the agent will be at the goal 
    use maze_instance.run(0) to show updates without running logic

    """
    
    start = (1, 1)
    end = (8,4)

    path,complexity = ucs_search(maze, start, end,neighbors)
    
    print("Time Complexity is: ",complexity)
    for pos in path:
        x, y = pos
        fringe = Fringe()
        maze_instance.add_thing(fringe, [x, y])
        fringe.color = PATH_COLOR

    if path is None:
        print("No path found!")
        return []
    
    actions = []
    for i in range(len(path) - 1):
        current = path[i]
        next = path[i+1]
        dx = next[0] - current[0]
        dy = next[1] - current[1]
        if dx == 0:
            if dy == -1:
                actions.append("up")
            elif dy == 1:
                actions.append("down")
        elif dy == 0:
            if dx == -1:
                actions.append("left")
            elif dx == 1:
                actions.append("right")
   
    return actions,complexity
   






In [30]:

# Add walls to the maze_instance
for i in range(len(maze_layout)):
    for j in range(len(maze_layout[0])):
        if maze_layout[i][j] == 1:
            maze_instance.add_thing(Wall(), [i, j])


agent = OurAgent(program)
goal = Goal()
start = [1,1]
end = [8,4]
maze_instance.add_thing(agent, start)
maze_instance.add_thing(goal, end)
maze_instance.exogenous_change()
it = -1
            
maze_instance.delete_thing(agent)
maze_instance.delete_thing(goal)

steps,complexity = algo(np.transpose(maze_layout))

maze_instance.add_thing(agent, start)
maze_instance.add_thing(goal, end)
maze_instance.run(len(steps))


print("Time Complexity is : ", complexity[0])
print('Space Complexity is : ', complexity[1])

,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,,,,,


Time Complexity is :  100
Space Complecity is :  99
